<a href="https://colab.research.google.com/github/Matheus-Fuzati-de-Carvalho/Matheus-Fuzati-de-Carvalho/blob/main/etl_vendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pipeline ETL - Resultados de Vendas

In [420]:
# Importando bibliotecas

# Autenticação
from google.colab import auth
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe
gc = gspread.authorize(creds)


# Manipulação e tratamento - Dados
import pandas as pd
from datetime import datetime, timedelta
import pytz

# Big Queryy
import pandas_gbq
from pandas_gbq import to_gbq
from google.cloud import bigquery
from google.oauth2 import service_account

In [421]:
# Desativando avisos
pd.options.mode.chained_assignment = None

In [422]:
# definindo função para abrir planilha
def abrirPlanilha(plan, pag):
  pagina = gc.open(plan)
  pagina = pagina.get_worksheet(pag).get_all_values()
  pagina = pd.DataFrame(pagina)
  pagina.columns = pagina.iloc[0]
  pagina = pagina[1:].reset_index(drop=True)
  return pagina

In [423]:
# Criando a variável com a data e hora de hoje para apresentar a data da atualização
dt = pytz.timezone('America/Sao_Paulo')
data_hora = datetime.now(dt).strftime("%d/%m/%Y, %H:%M:%S")
data =  data_hora.format("%d/%m/%Y")

## Extração de dados

In [424]:
# Vendas - Google Sheets
df_vendas = abrirPlanilha('vendas', 0)


# Clientes - Excel
df_clientes = pd.read_excel('/content/clientes.xlsx')


# Produtos - Big Query
credencial = service_account.Credentials.from_service_account_file(r'/content/cloud-engineer-444301-r6-a1e2ea45a24e.json',
                                                                   scopes=['https://www.googleapis.com/auth/bigquery'])


cliente = bigquery.Client(credentials=credencial, project=credencial.project_id)

consulta = """
SELECT
idProduto
, nomeProduto
, categoria
, precoCusto
, precoVenda
FROM `cloud-engineer-444301-r6.case_tools.produtos`
"""

produtos_job = cliente.query(consulta)

df_produtos = produtos_job.to_dataframe()

# Tratamento de dados

In [425]:
# Analisando o dataframe
df_produtos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30 entries, 0 to 29
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   idProduto    30 non-null     Int64  
 1   nomeProduto  30 non-null     object 
 2   categoria    30 non-null     object 
 3   precoCusto   30 non-null     float64
 4   precoVenda   30 non-null     float64
dtypes: Int64(1), float64(2), object(2)
memory usage: 1.3+ KB


In [426]:
# Tratando erros de dados
df_produtos = df_produtos.fillna('')
df_produtos = df_produtos.drop_duplicates()
df_produtos = df_produtos.dropna(how='all')

In [427]:
# Alterando as colunas de object para catgeory para diminuição do tamanho do dataframe
df_produtos.nomeProduto = df_produtos.nomeProduto.astype('category')
df_produtos.categoria = df_produtos.nomeProduto.astype('category')
df_produtos.idProduto = df_produtos.idProduto.astype('object')

In [428]:
# Criando a coluna de lucro
df_produtos['lucro'] = df_produtos.apply(lambda x: x['precoVenda'] - x['precoCusto'], axis=1)
df_produtos['att_em'] = data_hora

In [429]:
# Realizando o mesmo processo nos outros dataframes
df_vendas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   idVenda     10000 non-null  object
 1   idCliente   10000 non-null  object
 2   idProduto   10000 non-null  object
 3   dataVenda   10000 non-null  object
 4   valorVenda  10000 non-null  object
 5   regiao      10000 non-null  object
dtypes: object(6)
memory usage: 468.9+ KB


In [430]:
# Tratando erros de dados
df_vendas = df_vendas.dropna(how='all')
df_vendas = df_vendas.fillna('')
df_vendas = df_vendas.drop_duplicates()

In [431]:
df_vendas.regiao = df_vendas.regiao.astype('category')
df_vendas["valorVenda"] = pd.to_numeric(df_vendas["valorVenda"].str.replace(",", "."), errors="coerce")
df_vendas['dataVenda'] = pd.to_datetime(df_vendas['dataVenda'], format='%d/%m/%Y')

df_vendas['att_em'] = data_hora


In [432]:
# Tratando erros de dados
df_clientes = df_clientes.dropna(how='all')
df_clientes = df_clientes.fillna('')
df_clientes = df_clientes.drop_duplicates()

In [433]:
df_clientes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   idCliente     20 non-null     int64 
 1   nomeCliente   20 non-null     object
 2   emailCliente  20 non-null     object
 3   cidade        20 non-null     object
 4   estado        20 non-null     object
dtypes: int64(1), object(4)
memory usage: 928.0+ bytes


In [434]:
df_clientes.cidade =  df_clientes.cidade.astype('category')
df_clientes.estado =  df_clientes.estado.astype('category')
df_clientes.idCliente =  df_clientes.idCliente.astype('object')
df_clientes['att_em'] = data_hora


### Carregamento dos dados tratados

In [435]:
# Tabela Vendas Big query

to_gbq(
    dataframe=df_vendas,
    destination_table="case_tools.vendas",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",
    credentials=credencial,
)


100%|██████████| 1/1 [00:00<00:00, 759.15it/s]


In [436]:
# Tabela Produtos - Big Query

to_gbq(
    dataframe=df_produtos,
    destination_table="case_tools.produtos_tratada",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",
    credentials=credencial,  )


100%|██████████| 1/1 [00:00<00:00, 626.48it/s]


In [437]:
# Tabela Clientes - Big Query


to_gbq(
    dataframe=df_clientes,
    destination_table="case_tools.clientes",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",
    credentials=credencial,
)


100%|██████████| 1/1 [00:00<00:00, 1204.57it/s]


### Cruzando dados

In [438]:
# Padronizando colunas do join (merge)
df_vendas['idCliente'] = df_vendas['idCliente'].astype('int64')
df_vendas['idProduto'] = df_vendas['idProduto'].astype('int64')
df_clientes['idCliente'] = df_clientes['idCliente'].astype('int64')
df_produtos['idProduto'] = df_produtos['idProduto'].astype('int64')

In [439]:
# Realizando cruzamento
df =  pd.merge(df_vendas, df_clientes, on='idCliente', how='left')
df = pd.merge(df, df_produtos, on='idProduto', how='left')
df.drop(columns=['att_em_x', 'att_em_y'], inplace=True)

### Carregando os dados após o join

In [440]:
# Tabela Consolidada - Big Clientes

to_gbq(
    dataframe=df,
    destination_table="case_tools.vendas_consolidadas",
    project_id="cloud-engineer-444301-r6",
    if_exists="replace",
    credentials=credencial,
)


100%|██████████| 1/1 [00:00<00:00, 7928.74it/s]


# ETL  Finalizado